In [ ]:
from google.colab import drive
drive.mount('/content/drive')     # Allows drive access to colab

Mounted at /content/drive


In [ ]:
import gdown

# File url
file_id = "1HF6fB6_780fyNv5Tlxz0z_3V-A8p-aXP"
url = f"https://drive.google.com/uc?id={file_id}"

gdown.download(url, "fake.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1HF6fB6_780fyNv5Tlxz0z_3V-A8p-aXP
From (redirected): https://drive.google.com/uc?id=1HF6fB6_780fyNv5Tlxz0z_3V-A8p-aXP&confirm=t&uuid=6ab4e6a1-4a0f-484f-beca-8dbe72ff5530
To: /content/fake.zip
100%|██████████| 1.45G/1.45G [00:19<00:00, 74.1MB/s]


'fake.zip'

In [ ]:
import os
import zipfile
extract_path = "/content/extracted_fake"

# Create directory if it does not exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the zip file
with zipfile.ZipFile("fake.zip", "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete! Files are in:", extract_path)

Extraction complete! Files are in: /content/extracted_fake


In [ ]:
import glob
import numpy as np
import cv2
import copy

video_files =  glob.glob('/content/extracted_fake/*.mp4')

frame_count = []
for video_file in video_files:

  cap = cv2.VideoCapture(video_file)   # Opens the video file

  # Skip videos that have frames less than 150
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue

  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))   # Frame count

print("Frames:" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

Frames: [365, 478, 302, 445, 467, 611, 520, 307, 328, 322, 557, 409, 557, 400, 505, 446, 442, 530, 365, 491, 611, 322, 459, 470, 499, 497, 361, 365, 372, 428, 534, 422, 630, 412, 428, 505, 530, 491, 365, 409, 376, 422, 467, 361, 483, 611, 272, 376, 469, 499, 497, 272, 477, 412, 630, 294, 380, 520, 506, 483, 381, 303, 439, 459, 398, 376, 380, 557, 470, 409, 412, 398, 530, 380, 425, 458, 409, 458, 428, 372, 458, 520, 483, 326, 409, 497, 380, 491, 302, 302, 328, 529, 376, 328, 470, 326, 380, 611, 557, 505, 516, 372, 442, 611, 459, 445, 272, 381, 323, 303, 458, 294, 272, 322, 529, 322, 740, 372, 412, 529, 422, 516, 412, 303, 439, 505, 376, 307, 491, 372, 303, 464, 372, 483, 458, 383, 491, 470, 479, 439, 470, 557, 369, 459, 383, 400, 398, 425, 516, 409, 350, 294, 323, 326, 326, 323, 458, 380, 294, 372, 409, 469, 398, 499, 479, 529, 483, 445, 439, 458, 470, 294, 372, 400, 458, 516, 479, 380, 361, 323, 516, 491, 478, 442, 372, 412, 400, 458, 458, 303, 446, 520, 557, 499, 302, 422, 425, 400, 4

In [ ]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image  # Returns frame

In [ ]:
!pip install face_recognition
!mkdir '/content/drive/My Drive/preprocessedDataset'
!mkdir '/content/drive/My Drive/preprocessedDataset/fake'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=481aa882a2f6f9b8760edcd477d8017cec4a7fddd43c208a21ccfd9a160dca69
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models
mkdir: cannot create directory ‘/content/drive/My Drive/preprocessedDataset’: File exists
mkdir: cannot create directory ‘/content/drive/My Drive/preprocessedDataset/fake’: File exists


In [ ]:
import torch
import torchvision
# from torchvision import transforms
# from torch.utils.data import DataLoader
# from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
# import matplotlib.pyplot as plt
import face_recognition
from tqdm import tqdm

# process the frames
def create_face_videos(path_list,out_dir):

  # No. of video already preprocessed
  already_present_count = glob.glob(os.path.join(out_dir, '*.mp4'))

  print("No of videos already present " , len(already_present_count))

  # Skip already preprocessed videos
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)

    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue

    frames = []
    flag = 0
    face_all = []
    frames1 = []

    # Video writer object to convert frames to video
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))

    for idx,frame in enumerate(frame_extract(path)):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.face_locations(frame, model='hog')

          for i,face in enumerate(faces):
            if face:
              top,right,bottom,left = face
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass

          frames = []
    try:
      del top,right,bottom,left
    except:
      pass

    out.release()

print("Data preprocessed!")

Data preprocessed!


In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/preprocessedDataset/fake')

No of videos already present  1


  0%|          | 2/886 [00:16<2:08:41,  8.73s/it]

File Already exists:  /content/drive/My Drive/preprocessedDataset/fake/id4_id23_0006.mp4


100%|██████████| 886/886 [2:03:01<00:00,  8.33s/it]
